# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## load & save tensor

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## save

In [2]:
x = torch.arange(4)
torch.save(x, 'x.torch_tensor')

## load

In [3]:
x2 = torch.load('x.torch_tensor')
x2

tensor([0, 1, 2, 3])

## load & save a `list` of tensor

In [4]:
y = torch.zeros(4)
torch.save([x, y],'xy.torch_tensor')

In [5]:
x2, y2 = torch.load('xy.torch_tensor')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

## load & save a `dict` of tensor

In [6]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## load & save network params

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [8]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)
- **we need the same structure first!!!**

In [11]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval() # 处于predict模式 ， 处于训练模式时时.train() 
# 对Batch nomalization和dropout 有影响

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

## testing(check if the result is the same)

In [12]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])